In [19]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

In [20]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"  # 指定使用 GPU 3
import torch

In [21]:
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")
# import keras

import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset

import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification
%env TOKENIZERS_PARALLELISM=false

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizers.__version__: 0.19.1
transformers.__version__: 4.41.1
env: TOKENIZERS_PARALLELISM=false


In [22]:
import pandas as pd 
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments, 
    DataCollatorWithPadding
)
from datasets import Dataset
from glob import glob
import gc
import torch
from scipy.special import softmax

In [23]:
import spacy
nlp = spacy.load("en_core_web_sm")
with open('kaggle/input/english-word-hx/words.txt', 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)

## 数据load和处理

In [24]:

# keras.utils.set_random_seed(42) ##每次的随机值相同，方便复现
# keras.mixed_precision.set_global_policy("mixed_float16")



In [25]:
import polars as pl
columns = [  
    (
        pl.col("full_text").str.split(by="\n\n").alias("paragraph")
    ),
]
PATH = "kaggle/input/learning-agency-lab-automated-essay-scoring-2/"
TEST_DATA_PATH = "kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv"
MAX_LENGTH = 1024
EVAL_BATCH_SIZE = 1


## 添加模型

In [26]:
# Load model directlyfrom transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("addy88/argument-classifier")
model = AutoModelForSequenceClassification.from_pretrained("addy88/argument-classifier")

In [27]:
PATH = '/home/mcq/GitHub/aes2/kaggle/input/learning-agency-lab-automated-essay-scoring-2/'

train = pl.read_csv(PATH + "train.csv").with_columns(columns) 
test = pl.read_csv(PATH + "test.csv").with_columns(columns)

### 小型预处理


In [28]:
def cut(tmp):   
    tmp = tmp.with_columns(pl.col('full_text').str.slice(0, 513).alias
    ("cut"))
    return tmp

train = cut(train)

df =pd.DataFrame({ "cut": train['cut']})

In [29]:
args = TrainingArguments(
    ".", 
    per_device_eval_batch_size=1, 
    report_to="none"
    
)
def tokenize(sample):
    return tokenizer(sample['cut'], max_length=MAX_LENGTH, truncation=True)

trainer = Trainer(
        model=model, 
        args=args, 
        data_collator=DataCollatorWithPadding(tokenizer), 
        tokenizer=tokenizer
    )


In [30]:
ds = Dataset.from_pandas(df).map(tokenize).remove_columns(['cut'])

Map:   0%|          | 0/17307 [00:00<?, ? examples/s]

In [31]:
preds = trainer.predict(ds).predictions

In [32]:
predictions = []
predictions.append(softmax(preds, axis=-1))

In [42]:
print(predictions)
predictions_2d = np.array(predictions)
predictions_df2d = predictions_2d.reshape(-1, predictions_2d.shape[-1])
predictions_df2d.shape
predictions_df = pd.DataFrame(predictions_df2d)
# predictions_df.to_pickle('kaggle/input/argument-feat.pkl')

[array([[0.184659  , 0.815341  ],
       [0.86983293, 0.13016711],
       [0.12448   , 0.87552005],
       ...,
       [0.6726167 , 0.32738325],
       [0.8656717 , 0.13432834],
       [0.61259675, 0.38740325]], dtype=float32)]
